In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tfrecorder import TFrecorder
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%pylab inline

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Populating the interactive namespace from numpy and matplotlib


In [2]:
tfr = TFrecorder()
def input_fn_maker(path, data_info_path, shuffle=False, batch_size = 1, epoch = 1, padding = None):
    def input_fn():
        filenames = tfr.get_filenames(path=path, shuffle=shuffle)
        dataset=tfr.get_dataset(paths=filenames, data_info=data_info_path, shuffle = shuffle, 
                            batch_size = batch_size, epoch = epoch, padding =padding)
        iterator = dataset.make_one_shot_iterator()
        return iterator.get_next()
    return input_fn

padding_info = ({'image':[784,],'label':[]})
test_input_fn = input_fn_maker('mnist_tfrecord/test/',  'mnist_tfrecord/data_info.csv',batch_size = 512,
                               padding = padding_info)
train_input_fn = input_fn_maker('mnist_tfrecord/train/',  'mnist_tfrecord/data_info.csv', shuffle=True, batch_size = 128,
                               padding = padding_info)
train_eval_fn = input_fn_maker('mnist_tfrecord/train/',  'mnist_tfrecord/data_info.csv', batch_size = 512,
                               padding = padding_info)

In [3]:
def model_fn(features, mode):
    # reshape 784维的图片到28x28的平面表达，1为channel数
    features['image'] = tf.reshape(features['image'],[-1,28,28,1])
    # shape: [None,28,28,1]
    # create RNN cells:
    rnn_fcells = [tf.nn.rnn_cell.GRUCell(dim,kernel_initializer=tf.orthogonal_initializer) for dim in [128,256]]
    rnn_bcells = [tf.nn.rnn_cell.GRUCell(dim,kernel_initializer=tf.orthogonal_initializer) for dim in [128,256]]
    # stack cells for multi-layers RNN
    multi_rnn_fcell = tf.nn.rnn_cell.MultiRNNCell(rnn_fcells)
    multi_rnn_bcell = tf.nn.rnn_cell.MultiRNNCell(rnn_bcells)
    # create RNN layers
    ((outputs_fw, outputs_bw),(last_state_fw, last_state_bw)) = tf.nn.bidirectional_dynamic_rnn(
                                   cell_fw=multi_rnn_fcell,
                                   cell_bw=multi_rnn_bcell,
                                   inputs=tf.reshape(features['image'],[-1,28,28]),
                                   dtype=tf.float32)
    # shape: outputs: [None,28,256]
    # shape: last_state: [None,256]
    dense1 = tf.layers.dense(inputs=last_state_fw[1]+last_state_bw[1], units=1024, activation=tf.nn.relu, name= 'dense1')
    # shape: [None,1024]
    dropout = tf.layers.dropout(inputs=dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(inputs=dense1, units=10, name= 'output')
    # shape: [None,10]
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "labels": features['label'],
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
        }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=features['label'], logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=features['label'], predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [4]:
mnist_classifier = tf.estimator.Estimator(
    model_fn=model_fn, model_dir="mnist_model_birnn")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'mnist_model_birnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe7d248dc88>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
mnist_classifier.train(
    input_fn=train_input_fn)

read dataframe from mnist_tfrecord/train/train30000_31000.tfrecord x 55
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from mnist_model_birnn/model.ckpt-431
INFO:tensorflow:Saving checkpoints for 432 into mnist_model_birnn/model.ckpt.
INFO:tensorflow:loss = 0.114502, step = 432
INFO:tensorflow:global_step/sec: 5.82938
INFO:tensorflow:loss = 0.0570738, step = 532 (17.157 sec)
INFO:tensorflow:global_step/sec: 5.85307
INFO:tensorflow:loss = 0.18259, step = 632 (17.085 sec)
INFO:tensorflow:global_step/sec: 5.8075
INFO:tensorflow:loss = 0.0440512, step = 732 (17.218 sec)
INFO:tensorflow:global_step/sec: 5.8058
INFO:tensorflow:loss = 0.0487418, step = 832 (17.225 sec)
INFO:tensorflow:Saving checkpoints for 861 into mnist_model_birnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.068351.


In [6]:
eval_results = mnist_classifier.evaluate(input_fn=train_eval_fn)
print('train set')
print(eval_results)

read dataframe from mnist_tfrecord/train/train19000_20000.tfrecord x 55
INFO:tensorflow:Starting evaluation at 2018-04-07-13:58:43
INFO:tensorflow:Restoring parameters from mnist_model_birnn/model.ckpt-861
INFO:tensorflow:Finished evaluation at 2018-04-07-13:59:02
INFO:tensorflow:Saving dict for global step 861: accuracy = 0.981745, global_step = 861, loss = 0.0577625
train set
{'accuracy': 0.98174548, 'loss': 0.057762455, 'global_step': 861}


In [7]:
eval_results = mnist_classifier.evaluate(input_fn=test_input_fn)
print('test set')
print(eval_results)

read dataframe from mnist_tfrecord/test/test1000_2000.tfrecord x 10
INFO:tensorflow:Starting evaluation at 2018-04-07-13:59:05
INFO:tensorflow:Restoring parameters from mnist_model_birnn/model.ckpt-861
INFO:tensorflow:Finished evaluation at 2018-04-07-13:59:09
INFO:tensorflow:Saving dict for global step 861: accuracy = 0.9791, global_step = 861, loss = 0.0687892
test set
{'accuracy': 0.97909999, 'loss': 0.068789221, 'global_step': 861}
